In [56]:
import requests
from bs4 import BeautifulSoup as bs
import parse_methods
from parse_methods import parse_method
# from parse_tools import parse_tool
from parse_ingredients import parse_ingredient

Take in a URL

In [41]:
url = 'https://www.allrecipes.com/recipe/244716/shirataki-meatless-meat-pad-thai/'
page = requests.get(url)
soup = bs(page.content, "html.parser")

In [42]:
title = soup.find("h1", class_="headline heading-content elementFont__display").text

In [43]:
recipe_info = soup.find_all("div", class_="recipe-meta-item-header elementFont__subtitle--bold elementFont__transformCapitalize")
recipe_info = [x.text for x in recipe_info]
recipe_info_details = soup.find_all("div", class_="recipe-meta-item-body elementFont__subtitle")
recipe_info_details = [x.text[:-1] for x in recipe_info_details]

recipe_info = zip(recipe_info, recipe_info_details)
for item in recipe_info:
  print(item)

('prep:', '10 mins')
('cook:', '15 mins')
('total:', '25 mins')
('Servings:', '4')
('Yield:', '4 servings')


In [44]:
ingredients = soup.find_all("span", class_="ingredients-item-name elementFont__body")
# ingredients = [x.text for x in ingredients]
# need to translate some encodings and special characters
# also need to learn how to distinguish ingredients and measurements

In [45]:
# Source: https://stackoverflow.com/questions/15190930/python-beautifulsoup-parsing-html-fractions

import unicodedata as ud
import re

numerator = {
    'ONE':1,
    'TWO':2,
    'THREE':3,
    'FOUR':4,
    'FIVE':5,
    'SIX':6,
    'SEVEN':7,
    'EIGHT':8,
    'NINE':9,
    'ZERO':0,
    }

denominator = {
    'QUARTER':4,
    'HALF':2,
    'SEVENTH':7,
    'NINTH':9,
    'THIRD':3,
    'FIFTH':5,
    'SIXTH':6,
    'EIGHTH':8,
    'SIXTEENTH':16
    }

fraction = {}
for num in range(0x110000):
    s = chr(num)
    try:
        name = ud.name(s)
    except ValueError:
        continue
    match = re.search('FRACTION ({n}) ({d})'.format(
        n = '|'.join(numerator.keys()),
        d = '|'.join(denominator.keys()),
        ) , name)
    if match:
        fraction[num] = str(
           float(numerator[match.group(1)])/denominator[match.group(2)]).lstrip('0')
print(fraction)

{188: '.25', 189: '.5', 190: '.75', 2930: '.25', 2931: '.5', 2932: '.75', 2933: '.0625', 2934: '.125', 2935: '.1875', 3422: '.2', 3443: '.25', 3444: '.5', 3445: '.75', 3446: '.0625', 3447: '.125', 3448: '.1875', 8528: '.14285714285714285', 8529: '.1111111111111111', 8531: '.3333333333333333', 8532: '.6666666666666666', 8533: '.2', 8534: '.4', 8535: '.6', 8536: '.8', 8537: '.16666666666666666', 8538: '.8333333333333334', 8539: '.125', 8540: '.375', 8541: '.625', 8542: '.875', 8585: '.0', 11517: '.5', 43056: '.25', 43057: '.5', 43058: '.75', 43059: '.0625', 43060: '.125', 43061: '.1875', 68029: '.5', 68168: '.5', 69243: '.5', 69244: '.25', 69245: '.3333333333333333', 69246: '.6666666666666666', 69414: '.5', 73673: '.0625', 73674: '.0625', 73676: '.125', 73678: '.1875', 73679: '.2', 73680: '.25', 73681: '.5', 73682: '.5', 73683: '.75', 126125: '.25', 126126: '.5', 126127: '.75', 126268: '.5', 126269: '.16666666666666666'}


In [46]:
ingredients = [ingredient.text.strip().translate(fraction).encode('ascii', 'ignore').decode("utf-8") for ingredient in ingredients]
print(ingredients)

['2 (7 ounce) packages shirataki noodles, drained', '1 tablespoon vegetable oil', '1 (12 ounce) package tofu, cut into chunks', '.25 cup reduced-sodium soy sauce', '.5 cup lemon juice', '.25 cup white sugar', '2 tablespoons peanut butter', '1 tablespoon sriracha hot sauce', '2 eggs', '1 (4.5 ounce) can mushrooms', '.5 cup chopped cashews, divided', '1 cup bean sprouts', '1 lime, cut into wedges']


In [47]:
parsed_ingredients = []
for ingredient in ingredients:
    print(parse_ingredient(ingredient))
    parsed_ingredients.append(parse_ingredient(ingredient))

Doing split for 2  packages shirataki noodles, drained
Ingredient(name='packages shirataki noodles', quantity=2.0, unit='', comment='(7 ounce)  drained', original_string='2 (7 ounce) packages shirataki noodles, drained')
Doing split for 2  packages shirataki noodles, drained
Ingredient(name='vegetable oil', quantity=1.0, unit='tbsp', comment='', original_string='1 tablespoon vegetable oil')
Doing split for 1  package tofu, cut into chunks
Ingredient(name='package tofu', quantity=1.0, unit='', comment='(12 ounce)  cut into chunks', original_string='1 (12 ounce) package tofu, cut into chunks')
Doing split for 1  package tofu, cut into chunks
Ingredient(name='reduced-sodium soy sauce', quantity=0.25, unit='cup', comment='', original_string='.25 cup reduced-sodium soy sauce')
Ingredient(name='lemon juice', quantity=0.5, unit='cup', comment='', original_string='.5 cup lemon juice')
Ingredient(name='white sugar', quantity=0.25, unit='cup', comment='', original_string='.25 cup white sugar')
I

In [48]:
steps = soup.find_all("div", class_="section-body elementFont__body--paragraphWithin elementFont__body--linkWithin")
steps = [x.text[:-2] for x in steps]

In [49]:
nutrition = soup.find("div", class_="recipeNutritionSectionBlock")
nutrition = nutrition.text[1:-17]

nutrition

'Per Serving:  363 calories; protein 16.9g; carbohydrates 32.2g; fat 22.1g; cholesterol 93mg; sodium 1022.9mg.'

Methods

In [50]:
methods = soup.find_all("div", class_="paragraph")
directions = [method.text.strip().encode('ascii', 'ignore').decode("utf-8") for method in methods]
print(directions)

['Place shirataki noodles in a bowl of hot water; set aside.', 'Heat oil in a skillet over medium heat; cook and stir tofu until lightly browned and cooked through, 5 to 10 minutes. Transfer tofu to a plate, keeping skillet over heat.', 'Whisk soy sauce lemon juice, sugar, peanut butter, and sriracha sauce together in a small microwave-safe bowl; heat in microwave until peanut sauce is melted and smooth, about 20 seconds. Stir well.', 'Cook and stir eggs in the hot skillet until cooked through and scrambled, about 5 minutes. Mix mushrooms and tofu into eggs; add 1/2 of the peanut sauce and mix well.', 'Drain shirataki noodles and add to egg mixture; pour in the remaining peanut sauce. Sprinkle 1/4 cup cashews into tofu-noodle mixture and mix well. Reduce heat to medium-low and simmer until sauce thickens, 5 to 7 minutes.', 'Transfer tofu-noodle mixture to a serving bowl and top with remaining cashews, bean sprouts, and lime wedges.', 'Any type of meat alternative can be used in place o

In [51]:
methods = []
for i in range(len(directions)):
    method = parse_method(directions[i], i)
    methods.append(method)
    print(method.direction, method.primary_cooking, method.secondary_cooking)

Place shirataki noodles in a bowl of hot water; set aside. ['set'] ['place']
Heat oil in a skillet over medium heat; cook and stir tofu until lightly browned and cooked through, 5 to 10 minutes. Transfer tofu to a plate, keeping skillet over heat. ['keep'] ['cook', 'stir', 'brown', 'cook']
Whisk soy sauce lemon juice, sugar, peanut butter, and sriracha sauce together in a small microwave-safe bowl; heat in microwave until peanut sauce is melted and smooth, about 20 seconds. Stir well. ['stir'] ['sauce', 'melt']
Cook and stir eggs in the hot skillet until cooked through and scrambled, about 5 minutes. Mix mushrooms and tofu into eggs; add 1/2 of the peanut sauce and mix well. ['mix'] ['cook', 'stir', 'cook', 'mix', 'add']
Drain shirataki noodles and add to egg mixture; pour in the remaining peanut sauce. Sprinkle 1/4 cup cashews into tofu-noodle mixture and mix well. Reduce heat to medium-low and simmer until sauce thickens, 5 to 7 minutes. ['reduce'] ['drain', 'add', 'pour', 'remain', 

In [87]:
import spacy
sp = spacy.load('en_core_web_sm')

def parse_tool(direction):

    # sentences = direction.split(".")
    sentences = re.split('.;', direction)
    cleansentences = []
    for sentence in sentences:
        cleansentence = sentence.strip()
        cleansentence = cleansentence.lower()
        cleansentences.append(cleansentence)

    tools = []
    # print(cleansentences)
    for sentence in cleansentences:
        # print(sentence)
        spltsentence = sentence.split()
        cutword = None
        toollst = []
        for i in range(len(spltsentence)):
            keywords = ["in", "into", "using","use"]
            if spltsentence[i] in keywords:
                cutword = spltsentence[(i):]

        if cutword != None:
            print("cutword is: ", cutword)
            sen = sp(" ".join(cutword))
            
            for j in range(len(cutword)):
                if sen[j].pos_ == "VERB":
                    print("verb is: ", sen[j])
                    toollst.append(cutword[j])
                    break
                elif sen[j].pos_ == "NOUN":
                    if sen[j-1].pos_ == "ADJ":
                        print("noun is: ", sen[j], "adj is: ", sen[j-1])
                        toollst.append(cutword[j-1])
                        toollst.append(cutword[j])
                    else:
                        print("noun is: ", sen[j])
                        toollst.append(cutword[j])
                    break
                
            tool = " ".join(toollst)
            tools.append(tool)

    return tools

In [88]:
tools = []
for i in range(len(directions)):
    currtool = parse_tool(directions[i])
    tools.extend(currtool)

cleantools = []
while len(tools) > 0:
    tool = tools[0]
    in_ingredients = False
    for ingredient in parsed_ingredients:
        if tool in ingredient.name:
            in_ingredients = True
    if in_ingredients == False:
        cleantools.append(tool)
    tools.remove(tool)

print(cleantools)

cutword is:  ['in', 'a', 'bowl', 'of', 'hot', 'wate']
noun is:  bowl
cutword is:  ['in', 'a', 'skillet', 'over', 'medium', 'hea']
noun is:  skillet
cutword is:  ['in', 'a', 'small', 'microwave-safe', 'bow']
noun is:  microwave adj is:  small
cutword is:  ['in', 'microwave', 'until', 'peanut', 'sauce', 'is', 'melted', 'and', 'smooth,', 'about', '20', 'seconds.', 'stir', 'well.']
noun is:  microwave
cutword is:  ['into', 'egg']
noun is:  egg
cutword is:  ['into', 'tofu-noodle', 'mixture', 'and', 'mix', 'well.', 'reduce', 'heat', 'to', 'medium-low', 'and', 'simmer', 'until', 'sauce', 'thickens,', '5', 'to', '7', 'minutes.']
noun is:  tofu
cutword is:  ['in', 'place', 'of', 'the', 'tofu,', 'if', 'desired.']
noun is:  place
cutword is:  ['use', 'real', 'meat', 'or', 'regular', 'pasta.', "i'm", 'not', 'big', 'on', 'spice,', 'so', 'add', 'more', 'sriracha', 'for', 'a', 'hotter', 'dish.']
verb is:  use
['bowl', 'skillet', 'small microwave-safe', 'microwave', 'tofu-noodle', 'place', 'use']


In [89]:
sen = sp("in a small microwave-safe bowl")

In [94]:
sen[5].pos_

'ADJ'